In [1]:
import os
from apiclient.discovery import build

In [2]:
DEVELOPER_KEY = os.environ["COLLAB_CLIENT_KEY"]
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [3]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

In [4]:
sections = "snippet,statistics,topicDetails"

results = youtube.channels().list(part=sections, categoryId="GCR2FtaW5n", maxResults=50).execute()

In [10]:
results["items"][0]

{u'etag': u'"q5k97EMVGxODeKcDgp8gnMu79wM/LIiPCFFDqay9m9D73VsNClcfN1c"',
 u'id': u'UCxaVOVnhmT0-HCUv72jtOTA',
 u'kind': u'youtube#channel',
 u'snippet': {u'country': u'CA',
  u'customUrl': u'nordan',
  u'description': u"Welcome to my Channel!\n\nBe sure to connect with me on my Social Media's listed below!\n\n\nHave an awesome day people :D\n\nPO BOX ADDRESS -\n\nFaZe Clan (Nordan Shat) *\n1662 Old Country Road #40\nPlainview, NY\n11803\nUnited States\n\n* if meant for someone else put the other persons name instead of mine! But it better be for me.",
  u'localized': {u'description': u"Welcome to my Channel!\n\nBe sure to connect with me on my Social Media's listed below!\n\n\nHave an awesome day people :D\n\nPO BOX ADDRESS -\n\nFaZe Clan (Nordan Shat) *\n1662 Old Country Road #40\nPlainview, NY\n11803\nUnited States\n\n* if meant for someone else put the other persons name instead of mine! But it better be for me.",
   u'title': u'FaZe Rain'},
  u'publishedAt': u'2010-12-29T02:58:27.00

In [64]:
pages = 100
allTopics = set()
sections = "snippet,statistics,topicDetails"
df = pd.DataFrame()
nextPage = None

while pages > 0:

    results = youtube.channels().list(part=sections, categoryId="GCR2FtaW5n", maxResults=50, pageToken=nextPage).execute()

    title = []
    country = []
    subs = []
    videos = []
    views = []
    topics = []

    for channel in results["items"]:
        title.append(channel["snippet"]["title"])
        try:
            country.append(channel["snippet"]["country"])
        except:
            country.append('n/a')
        subs.append(int(channel["statistics"]["subscriberCount"]))
        videos.append(int(channel["statistics"]["videoCount"]))
        views.append(int(channel["statistics"]["viewCount"]))
        try:
            topics.append(channel["topicDetails"]["topicIds"])
            allTopics.update(channel["topicDetails"]["topicIds"])
        except:
            topics.append([])            

    d = {'title':title, 'country':country, 'subs':subs, 'videos':videos, 'views':views, 'topics':topics}    
    df = df.append(pd.DataFrame(d, columns=['title','country','subs','videos','views','topics']), ignore_index=True)
    try:
        nextPage = results['nextPageToken']
    except:
        break
    pages -= 1

In [71]:
df[df['subs'] < 10000]

,title,country,subs,videos,views,topics
829,aCeLiFyVS,n/a,0,1498,34393657,"[/m/0g5rv45, /m/01mw1, /m/03bt1gh, /m/0bzvm2, ..."
1045,HabibiGaming,n/a,0,40,10171278,"[/m/02psyd2, /m/09gltp8, /m/026wy8d, /m/02x9hs..."
1154,THUNDERONE,FI,0,735,12122540,"[/m/01mw1, /m/09wcn_, /m/03bt1gh, /m/010r5m26,..."
1253,SyndicateTwitch,GB,0,196,7955188,"[/m/0w_4m_c, /m/03bt1gh, /m/0hgpf7h, /m/0403l3..."
1258,Verticalex,US,0,2900,17559405,"[/m/04n3w2r, /m/03bt1gh, /m/011c4zs0, /m/0bzvm2]"
1259,waydong91,n/a,7420,295,1583634,"[/m/03bt1gh, /m/0bzvm2, /m/0k4j, /m/07yv9]"
1331,Necroscope86,n/a,0,5004,28839621,"[/m/0j7n6j0, /m/0403l3g, /m/0bzvm2]"


In [74]:
for topic in allTopics:
    df[topic] = 0

for i, channel in enumerate(df['topics']):
    for topic in channel:
        df[topic][i] = 1

/home/alex/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
len(df.columns)

532

In [56]:
results['nextPageToken']

u'COgHEAA'

In [57]:
df[['subs','views']].describe()

,subs,views
count,999.000000,9.990000e+02
mean,846002.205205,2.347479e+08
std,2004662.290321,6.732648e+08
min,0.000000,3.514706e+06
25%,170936.000000,3.374609e+07
50%,360183.000000,7.275103e+07
75%,762726.500000,1.855401e+08
max,43225860.000000,1.174814e+10


In [60]:
df[df['subs'] < 10000]

,title,country,subs,videos,views,topics,/m/01tn_k,/m/02cphr,/m/07d5v,/m/04h6hv,...,/m/0379fj,/m/01k8wb,/m/01jshl,/m/06zmzsz,/m/0kz2x1z,/m/010fcjp9,/m/04058p,/m/0bpsmq,/m/0ltv,/m/0134tlps
830,aCeLiFyVS,n/a,0,1498,34393657,"[/m/0g5rv45, /m/01mw1, /m/03bt1gh, /m/0bzvm2, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
lst = [1,2,3]
lst.append([])
lst

[1, 2, 3, []]

In [5]:
results = youtube.channels().list(part=sections, id="UCcx01rLXJ4dCtXqW3kPmBzg").execute()

In [76]:
df.to_csv('YTGaming_small.csv', encoding='utf-8')

In [6]:
results

{u'etag': u'"q5k97EMVGxODeKcDgp8gnMu79wM/Qdm0inzuUKyPlNpAhE1u7ZqqvMo"',
 u'items': [{u'etag': u'"q5k97EMVGxODeKcDgp8gnMu79wM/FlMdYlrPyRZp7pflxiiaw8CLJxE"',
   u'id': u'UCcx01rLXJ4dCtXqW3kPmBzg',
   u'kind': u'youtube#channel',
   u'snippet': {u'country': u'US',
    u'description': u"Hello, I'm LichWick. I like playing games and I like being weird. So I thought why not combine the two and put it on the internet for all to enjoy~ Or not. I can't tell you what to do. \n\nIf you like the flavor of my personality or the content of my videos or even if you have nothing else to do, be sure to subscribe for more!",
    u'localized': {u'description': u"Hello, I'm LichWick. I like playing games and I like being weird. So I thought why not combine the two and put it on the internet for all to enjoy~ Or not. I can't tell you what to do. \n\nIf you like the flavor of my personality or the content of my videos or even if you have nothing else to do, be sure to subscribe for more!",
     u'title': u'

In [13]:
df = pd.read_csv('YTGaming_small.csv')

In [16]:
df = df.drop('Unnamed: 0', axis=1)

In [40]:
pages = 100
allTopics = set()
sections = "snippet,statistics,topicDetails"
df = pd.DataFrame()
nextPage = None

while pages > 0:

    results = youtube.channels().list(part=sections, categoryId="GCQmVhdXR5ICYgRmFzaGlvbg", maxResults=50, pageToken=nextPage).execute()

    title = []
    country = []
    subs = []
    videos = []
    views = []
    topics = []

    for channel in results["items"]:
        title.append(channel["snippet"]["title"])
        try:
            country.append(channel["snippet"]["country"])
        except:
            country.append('n/a')
        subs.append(int(channel["statistics"]["subscriberCount"]))
        videos.append(int(channel["statistics"]["videoCount"]))
        views.append(int(channel["statistics"]["viewCount"]))
        try:
            topics.append(channel["topicDetails"]["topicIds"])
            allTopics.update(channel["topicDetails"]["topicIds"])
        except:
            topics.append([])            

    d = {'title':title, 'country':country, 'subs':subs, 'videos':videos, 'views':views, 'topics':topics}    
    df = df.append(pd.DataFrame(d, columns=['title','country','subs','videos','views','topics']), ignore_index=True)
    try:
        nextPage = results['nextPageToken']
    except:
        break
    pages -= 1

In [41]:
df.head()

,title,country,subs,videos,views,topics
0,Brooklyn and Bailey,n/a,2403641,165,289908236,"[/m/04rh6b, /m/0cr3d, /m/032tl, /m/019_rr]"
1,MyLifeAsEva,n/a,5963661,185,428463923,"[/m/032tl, /m/019_rr]"
2,AlexandrasGirlyTalk,US,1030872,118,92719643,"[/m/017rcq, /m/03q69, /m/014trl, /m/0wfztg3, /..."
3,Aspyn Ovard,n/a,2597132,277,147172042,"[/m/032tl, /m/019_rr]"
4,Manny Mua,US,1047196,131,37785287,"[/m/09zzb8, /m/014trl, /m/0157xl, /m/0wfztg3, ..."


In [42]:
for topic in allTopics:
    df[topic] = 0

for i, channel in enumerate(df['topics']):
    for topic in channel:
        df[topic][i] = 1

/home/alex/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [43]:
df.head()

,title,country,subs,videos,views,topics,/m/011q4634,/m/0m95s,/m/02cphr,/m/03glg,...,/m/0p57p,/m/0k65p,/m/0w0g3k2,/m/01268b,/m/0zdksk1,/m/025t5bz,/m/0ds4x,/m/05r655,/m/0_gv9ff,/m/0hhdb
0,Brooklyn and Bailey,n/a,2403641,165,289908236,"[/m/04rh6b, /m/0cr3d, /m/032tl, /m/019_rr]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MyLifeAsEva,n/a,5963661,185,428463923,"[/m/032tl, /m/019_rr]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AlexandrasGirlyTalk,US,1030872,118,92719643,"[/m/017rcq, /m/03q69, /m/014trl, /m/0wfztg3, /...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aspyn Ovard,n/a,2597132,277,147172042,"[/m/032tl, /m/019_rr]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Manny Mua,US,1047196,131,37785287,"[/m/09zzb8, /m/014trl, /m/0157xl, /m/0wfztg3, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df.to_csv('YouTube_Collab_Recommender/data/YTFashion_small.csv', encoding='utf-8')

In [45]:
pd.get_dummies(df, columns=['country'])

,title,subs,videos,views,topics,/m/011q4634,/m/0m95s,/m/02cphr,/m/03glg,/m/01mf0,...,/m/0hhdb,country_AU,country_CA,country_DK,country_GB,country_IE,country_IT,country_NZ,country_US,country_n/a
0,Brooklyn and Bailey,2403641,165,289908236,"[/m/04rh6b, /m/0cr3d, /m/032tl, /m/019_rr]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,MyLifeAsEva,5963661,185,428463923,"[/m/032tl, /m/019_rr]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,AlexandrasGirlyTalk,1030872,118,92719643,"[/m/017rcq, /m/03q69, /m/014trl, /m/0wfztg3, /...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Aspyn Ovard,2597132,277,147172042,"[/m/032tl, /m/019_rr]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,Manny Mua,1047196,131,37785287,"[/m/09zzb8, /m/014trl, /m/0157xl, /m/0wfztg3, ...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,Kandee Johnson,3593509,570,384228420,"[/m/014trl, /m/0wfztg3, /m/019_rr]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,Makeup Geek,1312507,629,173393876,"[/m/014trl, /m/01f43, /m/0wfztg3, /m/019_rr]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,Meredith Foster,4216300,323,297902718,"[/m/032tl, /m/019_rr]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,Simplynessa15,1229515,558,81100548,"[/m/032tl, /m/0wfztg3, /m/019_rr]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,MayBaby,4902662,293,393185739,"[/m/032tl, /m/019_rr]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
